In [ ]:
# долгота 37.3251553 широта 55.8686792

In [8]:

import pandas as pd

import requests
from bs4 import BeautifulSoup

import re
from random import randint


def stat(tag=0):
    url = 'https://www.worldometers.info/coronavirus/'
    website = requests.get(url).text
    soup = BeautifulSoup(website, 'lxml')
    table = soup.find_all('table')[tag]
    rows = table.find_all('tr')
    d = dict()
    for i in range(9):
        col = []
        key = rows[0].find_all('th')[i + 1].get_text().strip()

        for row in rows[1:230]:
            r = row.find_all('td')
            col.append(r[i + 1].get_text().strip())
        d[key] = col
    df = pd.DataFrame(d)
    df = df.rename(columns={'Country,Other': 'Country', 'Serious,Critical': 'SeriousCritical'})
    return df

In [9]:
stat()

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical
0,North America,"35,976,050","+6,069","821,845",+471,"27,678,760","+5,039","7,475,445","15,401"
1,South America,"21,315,538",+916,"557,137",+23,"19,001,143",+962,"1,757,258","20,993"
2,Asia,"28,716,562","+27,135","429,699",+151,"26,225,015","+7,862","2,061,848","25,743"
3,Europe,"39,946,313","+87,153","918,996","+1,858","28,196,861","+70,555","10,830,456","31,293"
4,Africa,"4,274,074",+335,"113,447",+4,"3,837,450",+263,"323,177","3,025"
...,...,...,...,...,...,...,...,...,...
224,Marshall Islands,4,,,,4,,0,
225,Samoa,3,,,,2,,1,
226,Vanuatu,3,,,,1,,2,
227,Micronesia,1,,,,1,,0,


In [ ]:
""""https://geocode-maps.yandex.ru/1.x
 ? geocode=<string>
 & apikey=<string>
 & [sco=<string>]
 & [kind=<string>]
 & [rspn=<boolean>]
 & [ll=<number>, <number>]
 & [spn=<number>, <number>]
 & [bbox=<number>,<number>~<number>,<number>]
 & [format=<string>]
 & [results=<integer>]
 & [skip=<integer>]
 & [lang=<string>]
 & [callback=<string>]
https://geocode-maps.yandex.ru/1.x/?apikey=ваш API-ключ&geocode=37.597576,55.771899
    """""
#https://geocode-maps.yandex.ru/1.x?apikey=8803784b-fa83-4b12-8060-37049f25ef5d&geocode=37.3251553,55.8686792

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
#определение адреса по координатам
url = "https://geocode-maps.yandex.ru/1.x"
params = {
     'apikey':'8803784b-fa83-4b12-8060-37049f25ef5d',
     'geocode':'37.3251553,55.8686792'
}
g = requests.get(url, params=params)
g.ok

True

In [ ]:
#определение адреса по координатам
url = "https://api-maps.yandex.ru/2.1/?lang=ru_RU&amp;apikey=<ваш API-ключ>"
params = {
     'lang':'ru_RU',
     'apikey':'ваш API-ключ'
     
}
g = requests.get(url, params=params)
g.ok

In [3]:
data = BeautifulSoup(g.text, features='xml')

In [27]:
print(data.prettify())

<?xml version="1.0" encoding="utf-8"?>
<ymaps xmlns="http://maps.yandex.ru/ymaps/1.x" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/gml http://schemas.opengis.net/gml/3.1.1/base/gml.xsd http://maps.yandex.ru/ymaps/1.x https://maps.yandex.ru/schemas/ymaps/1.x/ymaps.xsd http://maps.yandex.ru/geocoder/1.x http://maps.yandex.ru/schemas/geocoder/1.x/geocoder.xsd http://maps.yandex.ru/address/1.x http://maps.yandex.ru/schemas/search/1.x/address.xsd urn:oasis:names:tc:ciq:xsdschema:xAL:2.0 http://docs.oasis-open.org/election/external/xAL.xsd">
 <GeoObjectCollection>
  <metaDataProperty xmlns="http://www.opengis.net/gml">
   <GeocoderResponseMetaData xmlns="http://maps.yandex.ru/geocoder/1.x">
    <request>
     37.3251553,55.8686792
    </request>
    <found>
     9
    </found>
    <results>
     10
    </results>
    <Point xmlns="http://www.opengis.net/gml">
     <pos>
      37.325155 55.868679
     </pos>
    </Point>
   </GeocoderRespons

In [75]:
#вычисление границ запроса

"""Долгота. Тут все просто: длина окружности (меридиана) постоянна - 40 008,55 км, разделим на 360°, получим:
111,134861111     км в одном градусе, делим на 60 минут:
1,85224768519     км в одной минуте, делим на 60 секунд:
0,0308707947531 км (30,8707947531 м) в одной секунде.

Широта. Длина окружности различна - 40.075,696 км на экваторе, 0 на полюсах. Расчитывается как длина одного градуса на экваторе умноженного на косинус угла широты. Один градус на экваторе - 40 075,696 км / 360° = 111,321377778 км/° (111321,377778 м/°)

На примере Казани:
широта 55,79083°, cos(55,79083) = 0,56221574216 * 111321,377778 м/° = 62 586,631026062 м/°
62,586631026062 км в одном градусе, делим на 60 минут:
1,043110517 км в одной минуте, делим на 60 секунд:
0,017385175 км (17,385175 м) в одной секунде."""
#https://ru.stackoverflow.com/questions/1140347/%D0%9A%D0%B0%D0%BA-%D0%BE%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B8%D1%82%D1%8C-%D1%80%D0%B0%D0%B4%D0%B8%D1%83%D1%81-%D0%BA%D1%80%D1%83%D0%B3%D0%B0-%D0%BF%D0%BE%D0%B8%D1%81%D0%BA%D0%B0-%D0%B2-api-%D0%BF%D0%BE%D0%B8%D1%81%D0%BA-%D0%BF%D0%BE-%D0%BE%D1%80%D0%B3%D0%B0%D0%BD%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F%D0%BC

import math
shirvkm=1/111
print(shirvkm)
print(shirvkm*2)

a=math.cos(55.8686792)*111.3
dolvkm=1/a
print(dolvkm)
print(dolvkm*2)
#print(math.cos(55.8686792)*5)
#0.000011552511148424713

0.009009009009009009
0.018018018018018018
0.0115547300794101
0.0231094601588202


In [76]:
#определение пабов рядом
url="https://search-maps.yandex.ru/v1/"
params={
    'apikey':'0949a9ad-718d-4f48-ba42-a5d8845724cc',
    'text':'бар',
    'lang':'ru_RU',
    'll':'37.3251553,55.8686792',
    'spn':'0.018018018018018018,0.0231094601588202',
    'rspn':'1',
    'results':'10'
}
g = requests.get(url, params=params)
g.text

'{"type":"FeatureCollection","properties":{"ResponseMetaData":{"SearchResponse":{"found":3,"display":"multiple","boundedBy":[[37.3027685,55.8421738],[37.3387685,55.88835057]]},"SearchRequest":{"request":"бар","skip":0,"results":10,"boundedBy":[[37.31614629,55.85712274],[37.33416431,55.8802322]]}}},"features":[{"type":"Feature","geometry":{"type":"Point","coordinates":[37.322723,55.869698]},"properties":{"name":"На Рогах","description":"Кленовая ул., 1, жилой комплекс Микрогород в Лесу, посёлок Отрадное, Россия","boundedBy":[[37.3186175,55.86738993],[37.3268285,55.87200593]],"CompanyMetaData":{"id":"21439122404","name":"На Рогах","address":"Россия, Московская область, городской округ Красногорск, посёлок Отрадное, жилой комплекс Микрогород в Лесу, Кленовая улица, 1","Phones":[{"type":"phone","formatted":"+7 (916) 164-99-77"}],"Categories":[{"class":"malls","name":"Магазин пива"},{"class":"bars","name":"Бар, паб"}],"Hours":{"text":"ежедневно, 13:00–23:00","Availabilities":[{"Intervals":[

In [46]:
g_data = g.json()

In [47]:
g_data

{'type': 'FeatureCollection',
 'properties': {'ResponseMetaData': {'SearchResponse': {'found': 103,
    'display': 'multiple',
    'boundedBy': [[37.286313, 55.81547584], [37.361238, 55.89043117]]},
   'SearchRequest': {'request': 'бар',
    'skip': 0,
    'results': 3,
    'boundedBy': [[37.1751553, 55.7183883], [37.4751553, 56.01838831]]}}},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [37.322723, 55.869698]},
   'properties': {'name': 'На Рогах',
    'description': 'Кленовая ул., 1, жилой комплекс Микрогород в Лесу, посёлок Отрадное, Россия',
    'boundedBy': [[37.3186175, 55.86738993], [37.3268285, 55.87200593]],
    'CompanyMetaData': {'id': '21439122404',
     'name': 'На Рогах',
     'address': 'Россия, Московская область, городской округ Красногорск, посёлок Отрадное, жилой комплекс Микрогород в Лесу, Кленовая улица, 1',
     'Phones': [{'type': 'phone', 'formatted': '+7 (916) 164-99-77'}],
     'Categories': [{'class': 'malls', 'name': 'Маг

In [ ]:
#вычисление границ области просмотра в зависимости от км
